# Funciones anónimas o lambdas

Hemos definido las funciones como bloques de código etiquetados por un nombre y que pueden ser invocados por ese nombre.

No obstante hay casos en los que queremos enviar el bloque de código como función sin asignarle una etiqueta, esto es lo que se llama una **función anónima**. 

Python las define como sigue:

```
lambda arguments: expresion
```

Un posible ejemplo es el siguiente


In [0]:
doble = lambda x: x * 2
doble(5)

10

Esto no es más que otra forma de escribir lo siguiente

In [0]:
def otroDoble(x):
  return x * 2

print(doble(5))
print(otroDoble(5))

10
10


## Para que sirven las lambdas


Las funciones anónimas pueden servir como argumentos de [funciones de nivel superior](https://colab.research.google.com/drive/13hmw16rypbf0AmDVOeiUyKMmCz5M3BuZ#scrollTo=zdcNvAXh6ybd). 

Supongamos que necesitamos realizar la suma de todos los elementos de una lista. Podríamos crear una función que lo realizara, así

In [0]:
def sumaTodos(*listaValores):
    total = 0
    for valor in listaValores:
        total += valor
        
    return total
    
print(sumaTodos(2,5,8,9,7))

31


Supongamos que necesitamos transformar los valores antes de realizar la suma, por ejemplo calculando su cuadrado. Un enfoque clásico sería el siguiente, crear una nueva función que sume todos los cuadrados, así

In [0]:
def sumaTodosCuadrados(*listaValores):
  total = 0
  for valor in listaValores:
    total += valor ** 2
    
  return total

print(sumaTodosCuadrados(2,5,8,9,7))

223


Podría interesarnos separar las dos funcionalidades de cada una de ellas, lo que sería
- la suma de todos los valores y
- la transformación previa de dichos valores

Esto es relativamente sencillo si utilizamos [funciones de nivel superior](https://drive.google.com/open?id=13hmw16rypbf0AmDVOeiUyKMmCz5M3BuZ). 

Quedaría como sigue:


In [0]:
def normal(x):
    return x

def cuadrado(x):
    return x * x

def sumaTodos(f, *listaValores):
    total = 0
    for valor in listaValores:
        try:
            total = total + f(valor)
        except:
            return None
            
    return total
  

print(sumaTodos(normal, 2, 5, 8, 9, 7))
print(sumaTodos(cuadrado, 2, 5, 8, 9, 7))


31
223


El siguiente paso casi casi es automático. Imaginemos que lo que queremos es que cada elemento de la lista se transforme en su cubo.

Según el enfoque anterior deberíamos crear una función clásica nueva llamada `cubo(x)` y luego aplicar la funcion `sumaTodos`. 

Algo así:

In [0]:
def cubo(x):
  return x ** 3

print(sumaTodos(cubo, 2, 5, 8, 9, 7))

1717


La utilizacion de una función anónima (al fin y al cabo cubo va a ser una función de usar y tirar) nos facilita el trabajo. Veamos como quedaría la línea 4 del código anterior usando una lambda.

In [0]:
print(sumaTodos(lambda x: x**3, 2, 5, 8, 9, 7))

1717


Es más, ya podemos escribir directamente la transformación sin preocuparnos de crear una función con su nombre y parámetros, como sumar todos los valores cambiados de signo. Así

In [0]:
print(sumaTodos(lambda x: -x, 2, 5, 8, 9, 7))

-31


En definitiva las funciones anónimas o lambdas nos permiten codificar directamente funciones y utilizarlas como parámetros. Especialmente en el caso de que necesitemos funciones de usar y tirar, ahorrándonos el tener que definirlas en otra parte del código.

## map, filter, reduce

El enfoque planteado para justificar las lambdas en los párrafos anteriores (la transformación de listas de elementos) es una situación habitual en la programación de aplicaciones en las que se producen flujos de datos. Estos datos deben ser procesados (eso es lo que hacen los programas) y esas transformaciones son siempre las mismas. 

Imagina esta situación, tenemos lectores de datos de la calidad del aire que envían cada vez que consiguen conexión (o cada 30 minutos si la conexión es continua) datos de la calidad del aire según la siguiente trama.



<img src="https://docs.google.com/uc?export=download&id=1SfrXXNeYvrmZj7Pr0TTaV6iJIkNEuMk2" width="600">



- Fecha (yyyymmdd): Caracteres 1 - 8 
- Hora (HHMMSSmmmmmm):  Caracteres 9 - 21
- Modelo Lector (XXX): Caracteres 22 - 24
- Datos de calidad del aire: Caracteres 24- 224
- En función del número de lector. Los siguientes datos iran en tramas distintas. Por ejemplo
  - número c23:
    - polen: Caracteres 1 - 6 según el formato 4, 2 (por ejemplo 89,21 se escribirá 008921)
    - NOx: Caracteres 7-12 formato 4,2
    - 03: Caracteres 13-18 formato 4,2
  - número c21:
    - NOx: Caracteres 1-5 segun el formato 3,2
    - polen: Caracteres 6-10 según el formato 2,2
  
  ...
  
Cada vez que recibamos una medida deberemos hacer dos transformaciones. Una común que reciba fecha, hora y número del lector. Y la segunda transformación dependerá del número del lector

Debemos construir un código que reciba una lista de medidas y que para cada medida haga
1. Obtención de datos generales 
2. Obtención de parámetros del aire

Algo así:

Primero definimos las funciones de procesamiento de trama, serán tres, una por cada transformación:
- tramageneral
- modelo c21
- modelo c23

In [0]:
'''
  Funciones de procesamiento de tramas
'''

def c23(data):
  resultado = {
      'polen': float(data[:4] + "." + data[4:6]),
      'NOx': float(data[6:10] + "." + data[10:12]),
      'O3': float(data[12:16] + "." + data[16:18])
  }
  return resultado

def c21(data):
  resultado = {
      'NOx': float(data[:3] + "." + data[3:5]),
      'polen': float(data[5:7] + "." + data[7:9])
  }
  return resultado

lectores = {}
lectores['c21']=c21
lectores['c23']=c23

def tramageneral(t):
  resultado = {
      'año': int(t[:4]),
      'mes': int(t[4:6]),
      'dia': int(t[6:8]),
      'hh': int(t[8:10]),
      'mm': int(t[10:12]),
      'ss': int(t[12:14]),
      'milli': int(t[14:20]),
      'matricula': t[20:23],
      'data': t[23:]
  }
  
  if resultado['matricula'] in lectores:
    resultado['data'] = lectores[resultado['matricula']](resultado['data'])
  return resultado



En segundo lugar el proceso que recibe las tramas y que las procesa en su totalidad

In [0]:
'''
  Recepción de tramas
'''
def procesaLecturas(*listaMedidas):
  resultado = []
  for medida in listaMedidas:
    fMedida = tramageneral(medida)
    resultado.append(fMedida)
  return resultado



Recibamos ahora algunas tramas...

In [0]:
'''
  Ejemplo de funcionamiento
'''
tramaNuevosMinisterios = "20181222132145098906c23008921978323003456"
tramaSol = "20181221001726365729c21023994534"
tramaColon = "20190101022356986345c23012345654321975310"

procesaLecturas(tramaNuevosMinisterios, tramaSol, tramaColon)

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 22,
  'hh': 13,
  'matricula': 'c23',
  'mes': 12,
  'milli': 98906,
  'mm': 21,
  'ss': 45},
 {'año': 2018,
  'data': {'NOx': 23.99, 'polen': 45.34},
  'dia': 21,
  'hh': 0,
  'matricula': 'c21',
  'mes': 12,
  'milli': 365729,
  'mm': 17,
  'ss': 26},
 {'año': 2019,
  'data': {'NOx': 6543.21, 'O3': 9753.1, 'polen': 123.45},
  'dia': 1,
  'hh': 2,
  'matricula': 'c23',
  'mes': 1,
  'milli': 986345,
  'mm': 23,
  'ss': 56}]

Si ejecutamos ahora procesaLecturas

In [0]:
procesaLecturas(tramaNuevosMinisterios, tramaSol)

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 13,
  'hh': 21,
  'matricula': 'c23',
  'mes': 1222,
  'milli': 98906,
  'mm': 45},
 {'año': 2018,
  'data': {'NOx': 23.99, 'polen': 45.34},
  'dia': 0,
  'hh': 17,
  'matricula': 'c21',
  'mes': 1221,
  'milli': 365729,
  'mm': 26}]

Al final lo que hacemos es procesar cada trama aplicando una función (con su lógica condicional interna). Podríamos haberlo hecho utilizando el operador map

In [0]:
resultado = map(tramageneral, listaMedidas)
list(resultado)

[{'año': 2018,
  'data': {'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21},
  'dia': 13,
  'hh': 21,
  'matricula': 'c23',
  'mes': 1222,
  'milli': 98906,
  'mm': 45},
 {'año': 2018,
  'data': {'NOx': 23.99, 'polen': 45.34},
  'dia': 0,
  'hh': 17,
  'matricula': 'c21',
  'mes': 1221,
  'milli': 365729,
  'mm': 26}]

In [0]:
list(modC23)

[{'NOx': 9783.23, 'O3': 34.56, 'polen': 89.21}]

In [0]:
from functools import reduce
print(reduce(lambda x,y: x + y, range(101)))
print(reduce(lambda x,y: x + (lambda x: x*x)(y), range(101)))

5050
338350


In [0]:
list(filter(lambda x: x%2 == 0, range(11)))

[0, 2, 4, 6, 8, 10]